In [ ]:
# Julia version code
"""
nonnegative linear regression 
"""
# Discussion from Dec 22
# 1. Currently, our runtime is O(n/sqrt(eps)*(m+n)). The per iteration O(m)
# is unavoidable; however, IF we are NOT required to output the optimizer xtilde_ktotal, 
# then by maintaining 1^T x in each iteration (at a cost of O(1)), we can completely
# avoid O(n) per iteration. 

using LinearAlgebra, BenchmarkTools, Plots, Convex, SCS, NonNegLeastSquares, MLDatasets


In [1]:
function alg_ours_with_restart(C::Matrix{Float64}, b::Matrix{Float64}, ϵ::Float64 )
    
    extra_term_nnls = 0.5*norm(b)^2
    m, n = size(C)
    K = ceil(log2(1/ϵ))
    col_norm = norm.(eachcol(C))
    inv_col_norm_square = 1.0 ./(col_norm.^2)
    idx_seq = 1:n
    
    x0 = zeros(n)
    y0 = zeros(m)
    z0 = zeros(m)
    
    for i=1:K
        
        xktilde, yktilde, zktilde, sumxtildek = alg_our_core(x0, y0, z0, sumx0, parameter)
        x0[:] = xktilde[:]
        y0[:] = yktilde[:]
        z0[:] = zktilde[:]
        sumx0 = sumxtildek
        
    end
    return(func_value)
end

function alg_our_core(x0, y0, z0, sumx0, parameter)
        # reset all the scaling factors 
        previous_A = 1.0/n
        previous_a = previous_A #a_1, A_1
        a = 1.0/(n*n) # a_2
        A = (n+1.0)/(n * n) # A_2
        
        # compute x1 using the input x0 
        # we redefined phio(x) = 1/2 * ||x-x0||_A^2, hence updating x requires x0 
        p = copy(x0) 
        x = copy(x0)
        j = rand(idx_seq)
        p[j] += inv_col_norm_square[j]
        x[j] = min(inv_col_norm_square[j], max(0, p[j])) #x and x0 differ only at j 
        
        # compute y1
        previous_y = copy(y0)
        z = copy(z0) 
        z += C[:, j] * (x[j] - x0[j]) # z_1 = A x_1 = A (x_0 + (x_1 - x_0))
        y = copy(z) # y_1 = A xtilde1 = A x_1 = z_1 
    
        # compute ȳ, ỹ (because we need to return it), and some auxiliary variables 
        ȳ = zeros(m) # arbitrary initialization 
        ȳ[:] = y[:] + previous_a/a * (y[:] - previous_y[:]) #ybar_1 
        s = zeros(n) # need this so that xtildek = xk + sk/Ak; s_1
        u = zeros(m) # u_k = A*s_k; we maintain this so we can cheaply return A xtildek
        ỹ = copy(y) # ytildek = convex comb of yi's, so ytilde1 = y1
    
        # func_value = 0
        func_value=zeros(Int(ceil(K/n)))
    
        # restart value init; ||Axitlde_ktotal||^2 - 1^T xtilde_ktotal 
        restart_val_prev = -sumx0+ 0.5* norm(z0)^2 +0.5*norm(y0)^2
        restart_val_curr = restart_val_prev
        sumx0 += x[j] - x0[j]
        sums1 = 0 
        
        while (restart_val_curr >= 0.5*restart_val_prev)
            # updates related to x
            j = rand(idx_seq)
            p[j] += - n * inv_col_norm_square[j] * a * (sum(C[:,j] .* ȳ) - 1)
            prev_xj = x[j]
            x[j] = min(inv_col_norm_square[j], max(0, p[j]))
            # update s so that we may return xtildek at only O(1) cost
            s[j] += ((n-1) * a -  previous_A) * (x[j] - prev_xj)
        
            # updates related to y 
            previous_y[:] = y[:]
            z[:] += C[:, j] * (x[j] - prev_xj)
            u[j] += ((n-1) * a -  previous_A) * (x[j] - prev_xj) * C[:, j]    
            y[:] = previous_A/A * y[:] + a/A * z[:] + (n-1) * a/A * (x[j] - prev_xj) * C[:,j]
            # need to update ytilde each time because that's what we want to return, 
            # and we aren't saving all the yi's. 
            ỹ[:] = previous_A/A * ỹ[:] + a/A * y[:]
        
            # update sums of xtilde and s to check the restart condition 
            sumx0 += (1 + ((n-1)*a - previous_A)/A)*(x[j] - prev_xj)- (a/(previous_A*A))*sums1
            sums1 += ((n-1)*a - previous_A)*(x[j] - prev_xj)
        
            # update scaling factors 
            previous_a, previous_A = a, A
            a = min(n * a/(n-1), sqrt(A)/(2*n))
            A += a
        
            # update ȳ (note that ȳ_k depends on a_k and a_{k+1})
            ȳ[:] = y[:] + previous_a/a * (y[:] - previous_y[:])
        
            # compute the restart condition 
            restart_val_curr = -sumx0+ 0.5* norm(z+ (1.0/previous_A) * u)^2 +0.5*norm(ỹ)^2
        end
        return x + (1.0/previous_A) * s, ỹ, z+ (1.0/previous_A) * u, sumx0 
end

alg_ours (generic function with 1 method)

In [2]:
a = [1 2 5 78]
b = copy(a)

1×4 Matrix{Int64}:
 1  2  5  78

In [3]:
b[3] = -5

-5

In [4]:
b

1×4 Matrix{Int64}:
 1  2  -5  78

In [5]:
a

1×4 Matrix{Int64}:
 1  2  5  78